# PREDICTION WEB APPLICATION

### Importing liabriess 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import os
import glob
import json 
import requests
from bs4 import BeautifulSoup
%matplotlib inline

#### Creating function for Downloading Data For Training and Testing The Model

In [2]:
# os.listdir('./Prediction_train&test_Data')
def load_data(url):
    
    folderName = './Prediction_train&test_Data/'
    fileName = 'league_data.json'
    path = os.path.join(folderName, fileName)

    if os.path.exists(path):
        files = glob.glob('./Prediction_train&test_Data/*')
        for f in files:
            os.remove(f)

    response = requests.get(url)
    # print(path)

    with open(path, 'wb') as outfile:
        outfile.write(response.content)
     

        
url = 'https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14001'
load_data(url)

#### Reading Json Files 

In [3]:
folderName = './Prediction_train&test_Data/'
fileName = 'league_data.json'
path = os.path.join(folderName, fileName)

with open(path, 'r') as outfile:
    json_obj = json.load(outfile)

In [4]:
clubes = json_obj['teams']
matches = json_obj['matches']
teams = []
resulte = []
wonmarke = []

count = 0
for key, val in matches.items():
    
    teams.append({key: []})
    resulte.append({key: []})
    wonmarke.append({key: []})
    
    for x in val:
        
        teams[count][key].append([val[x]['teamA'], val[x]['teamB']])
        resulte[count][key].append(val[x]['result'])
        wonmarke[count][key].append(val[x]['wonmarkets'])
        
    count +=1

In [5]:
# json_obj
# teams
# resulte
# wonmarke

#### Wranggling Data into Data Base

- Data
    - columns:
        - teams
        - winners(0 home, 1 draw, 2 away)



In [6]:
# resulte

In [7]:
# wonmarke
# teams

In [8]:
# events = []
events = {}
teams_index = {}
for x in range(len(teams)):
    for y, obj in teams[x].items():
        events[y] = []
        teams_index[y] = []
        for i in obj:
            teamA = clubes[i[0]]['fifa']
            teamB = clubes[i[1]]['fifa']
            events[y].append(f'{teamA}-{teamB}')
            teams_index[y].append('-'.join(i))

In [9]:
score = {}
for x in range(len(wonmarke)):
    for y, string in wonmarke[x].items():
        score[y] = []
        for j in string:
            resulte_win = j.split(',')[0]
            score[y].append(resulte_win)

In [10]:
# score
# events
socore_dataframe = pd.DataFrame(score)
events_dataframe = pd.DataFrame(events)
teams_index_dataframe = pd.DataFrame(teams_index)

In [11]:
socore_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Home,Home,Draw,Away,Draw,Draw,Draw,Home,Away,Draw,Home,Home,Draw,Away,Home
1,Draw,Home,Draw,Draw,Away,Away,Away,Home,Draw,Home,Away,Home,Home,Away,Away
2,Home,Away,Away,Away,Away,Draw,Home,Draw,Home,Draw,Home,Draw,Draw,Away,Away
3,Away,Draw,Draw,Away,Home,Home,Home,Home,Home,Draw,Home,Home,Draw,Draw,Away
4,Home,Home,Home,Away,Home,Away,Home,Draw,Away,Draw,Draw,Away,Away,Home,Home


In [12]:
events_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,SOU-BRI,BRI-MNC,CHE-LEI,ARS-CRY,BUR-WAT,TOT-BUR,WHU-MNU,ASV-BRI,NWC-CHE,LEE-WOL,LIV-WAT,ASV-WAT,MNC-NWC,WHU-NOR,BRI-EVE
1,NWC-LIV,LEI-MNU,NWC-TOT,LEI-WHU,SOU-LEE,ASV-ARS,NWC-EVE,LIV-ARS,NOR-LEI,TOT-ARS,MNC-CHE,MNU-NOR,LIV-LEE,ASV-TOT,WOL-CHE
2,WHU-WAT,ASV-CRY,MNC-LIV,LEE-NWC,NOR-LIV,LEI-SOU,CRY-WAT,LEE-NOR,BRI-LIV,BRN-NOR,ARS-LEE,EVE-CRY,SOU-ASV,MNU-WOL,WAT-LEI
3,BUR-WOL,ARS-BUR,WHU-LEE,BRN-CHE,CHE-EVE,BRN-NWC,NOR-TOT,CHE-WHU,WOL-TOT,LEI-CRY,NOR-EVE,BRN-WOL,BUR-WHU,SOU-MNC,NOR-NWC
4,EVE-LEI,TOT-WHU,BUR-BRI,WAT-MNC,WHU-BRN,MNU-CHE,MNC-LEI,TOT-CRY,SOU-MNU,MNU-MNC,BRI-TOT,TOT-LIV,CRY-MNU,LEI-LIV,MNC-ASV


In [13]:
teams_index_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,52-71,71-46,47-63,48-55,64-70,51-64,58-50,60-71,53-47,827-525,45-70,60-70,46-53,58-66,71-49
1,53-45,63-50,53-51,63-58,52-827,60-48,53-49,45-48,66-63,51-48,46-47,50-66,45-827,60-51,525-47
2,58-70,60-55,46-45,827-53,66-45,63-52,55-70,827-66,71-45,972-66,48-827,49-55,52-60,50-525,70-63
3,64-525,48-64,58-827,972-47,47-49,972-53,66-51,47-58,525-51,63-55,66-49,972-525,64-58,52-46,66-53
4,49-63,51-58,64-71,70-46,58-972,50-47,46-63,51-55,52-50,50-46,71-51,51-45,55-50,63-45,46-60


In [14]:
def melt_table(dataframe):
    dataframe['index'] = 1
    values_vars = [str(x) for x in range(1, dataframe.shape[1])]
    resulte = pd.melt(dataframe, id_vars=['index'], value_vars=values_vars)
    return resulte

In [15]:
new_events_df = melt_table(events_dataframe).rename(columns={'value': 'events', 'variable': 'weeks'})[['events','weeks']]
new_score_df = melt_table(socore_dataframe).rename(columns={'value': 'score', 'variable': 'weeks'})[['score','weeks']]
teams_index_df = melt_table(teams_index_dataframe).rename(columns={'value': 'teams_index', 'variable': 'weeks'})[['teams_index','weeks']]

In [16]:
new_events_df.head()

,events,weeks
0,SOU-BRI,1
1,NWC-LIV,1
2,WHU-WAT,1
3,BUR-WOL,1
4,EVE-LEI,1


In [17]:
new_score_df.head()

,score,weeks
0,Home,1
1,Draw,1
2,Home,1
3,Away,1
4,Home,1


In [18]:
teams_index_df.head()

,teams_index,weeks
0,52-71,1
1,53-45,1
2,58-70,1
3,64-525,1
4,49-63,1


In [19]:
events_score_df = pd.concat([new_events_df, new_score_df, teams_index_df], axis=1, join="inner").iloc[:, np.r_[0:2, 2, 4]] # [['events', 'score', 'teams_index']]

In [20]:
events_score_df.head(6)

,events,weeks,score,teams_index
0,SOU-BRI,1,Home,52-71
1,NWC-LIV,1,Draw,53-45
2,WHU-WAT,1,Home,58-70
3,BUR-WOL,1,Away,64-525
4,EVE-LEI,1,Home,49-63
5,CHE-TOT,1,Home,47-51


#### Saving Data Into CSV file

In [21]:
events_score_df.to_csv('./Prediction_train&test_Data/premier_leage_data.csv', index=False)